In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

from functions import *

pd.set_option('display.max_columns', 100)

raw

In [2]:
target = pd.read_csv('processed-data/target.csv', index_col=0)
target

,TARGET
SK_ID_CURR,
0,0
1,1
2,0
3,0
4,0
...,...
307505,0
307506,0
307507,0


In [3]:
installments = pd.read_csv('dseb-63-data-preparation-final-project/dseb63_final_project_DP_dataset/dseb63_final_project_DP_dataset/dseb63_installments_payments.csv')
installments.head()

,SK_ID_PREV,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT,SK_ID_CURR
0,1054186,1.0,6,-1180.0,-1187.0,6948.360,6948.360,147397.0
1,2452854,1.0,21,-546.0,-552.0,11302.605,11302.605,147397.0
2,1054186,1.0,2,-1300.0,-1307.0,6948.360,6948.360,147397.0
3,1682318,1.0,2,-240.0,-243.0,7374.510,7374.510,147397.0
4,2452854,1.0,10,-876.0,-882.0,11302.605,11302.605,147397.0


In [4]:
# join target and installments
installments.sort_values(by=['SK_ID_PREV', 'NUM_INSTALMENT_VERSION'], inplace=True)
installments = installments.merge(target, on='SK_ID_CURR', how='left')
installments.head(20)

,SK_ID_PREV,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT,SK_ID_CURR,TARGET
0,1000001,1.0,1,-268.0,-294.0,6404.310,6404.310,117953.0,0.0
1,1000001,2.0,2,-238.0,-244.0,62039.115,62039.115,117953.0,0.0
2,1000003,1.0,3,-34.0,-49.0,4951.350,4951.350,6707.0,0.0
3,1000003,1.0,1,-94.0,-108.0,4951.350,4951.350,6707.0,0.0
4,1000003,1.0,2,-64.0,-81.0,4951.350,4951.350,6707.0,0.0
5,1000004,1.0,6,-712.0,-752.0,3391.110,3391.110,187620.0,NaN
6,1000004,1.0,5,-742.0,-752.0,3391.110,3391.110,187620.0,NaN
7,1000004,1.0,2,-832.0,-851.0,3391.110,3391.110,187620.0,NaN
8,1000004,1.0,3,-802.0,-830.0,3391.110,3391.110,187620.0,NaN
9,1000004,1.0,4,-772.0,-830.0,3391.110,3391.110,187620.0,NaN


eda

In [5]:
installments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7744758 entries, 0 to 7744757
Data columns (total 9 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   SK_ID_PREV              int64  
 1   NUM_INSTALMENT_VERSION  float64
 2   NUM_INSTALMENT_NUMBER   int64  
 3   DAYS_INSTALMENT         float64
 4   DAYS_ENTRY_PAYMENT      float64
 5   AMT_INSTALMENT          float64
 6   AMT_PAYMENT             float64
 7   SK_ID_CURR              float64
 8   TARGET                  float64
dtypes: float64(7), int64(2)
memory usage: 531.8 MB


In [6]:
installments.describe()

,SK_ID_PREV,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT,SK_ID_CURR,TARGET
count,7.744758e+06,7.744758e+06,7.744758e+06,7.744758e+06,7.743119e+06,7.744757e+06,7.743119e+06,7.744757e+06,6.190746e+06
mean,1.902824e+06,8.254635e-01,1.991675e+01,-1.055681e+03,-1.064303e+03,1.672476e+04,1.685389e+04,1.535739e+05,7.654215e-02
std,5.360570e+05,9.774997e-01,2.763188e+01,8.009713e+02,8.006232e+02,4.997746e+04,5.407602e+04,8.864267e+04,2.658636e-01
min,1.000001e+06,0.000000e+00,1.000000e+00,-2.922000e+03,-3.160000e+03,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,1.432108e+06,0.000000e+00,4.000000e+00,-1.677000e+03,-1.686000e+03,4.050000e+03,3.375000e+03,7.684800e+04,0.000000e+00
50%,1.895642e+06,1.000000e+00,9.000000e+00,-8.420000e+02,-8.510000e+02,8.611920e+03,7.906185e+03,1.530760e+05,0.000000e+00
75%,2.368895e+06,1.000000e+00,2.100000e+01,-3.690000e+02,-3.790000e+02,1.641312e+04,1.575000e+04,2.301440e+05,0.000000e+00
max,2.843495e+06,1.780000e+02,2.750000e+02,-2.000000e+00,-2.000000e+00,3.771488e+06,3.771488e+06,3.075080e+05,1.000000e+00


In [7]:
# # plot correlation matrix
# corr = installments.corr()
# plt.figure(figsize=(10, 10))
# sns.heatmap(corr, annot=True, fmt='.2f', cmap='coolwarm')
# plt.show()

In [8]:
# # plot histogram
# for col in installments.columns:
#     if col not in ['SK_ID_CURR', 'TARGET', 'SK_ID_PREV']:
#         fig, ax = plt.subplots(1, 2, figsize=(12, 4))
#         ax[0].hist(installments[col], bins=30)
#         ax[0].set_title(col)
#         ax[1].scatter(installments[col], installments['TARGET'])
#         ax[1].set_title(col)
#         plt.show()

fill na

In [9]:
# fill missing values with mode
for col in installments.columns:
    installments[col].fillna(installments[col].mode()[0], inplace=True)

installments.isnull().sum()

SK_ID_PREV                0
NUM_INSTALMENT_VERSION    0
NUM_INSTALMENT_NUMBER     0
DAYS_INSTALMENT           0
DAYS_ENTRY_PAYMENT        0
AMT_INSTALMENT            0
AMT_PAYMENT               0
SK_ID_CURR                0
TARGET                    0
dtype: int64

In [10]:
# # plot histogram hue by target
# for col in installments.select_dtypes('number').columns:
#     if not col in ['SK_ID_CURR', 'TARGET', 'SK_ID_PREV']:
#         plt.figure(figsize=(12, 6))
#         sns.kdeplot(installments.loc[installments['TARGET'] == 0, col], label='target == 0')
#         sns.kdeplot(installments.loc[installments['TARGET'] == 1, col], label='target == 1')
#         plt.xlabel(col)
#         plt.ylabel('Density')
#         plt.title(f'{col} Distribution')
#         plt.legend()
#         plt.show()

feature engineering

In [11]:
# Change in Installment Version
installments['VERSION_CHANGE'] = installments.groupby('SK_ID_PREV')['NUM_INSTALMENT_VERSION'].diff().fillna(0)

# Installment Timing
installments['TIMING_DIFF'] = installments['DAYS_ENTRY_PAYMENT'] - installments['DAYS_INSTALMENT']

# Payment Ratio
installments['PAYMENT_RATIO'] = installments['AMT_PAYMENT'] / installments['AMT_INSTALMENT']

# Trend in Payment Amounts
installments['MOVING_AVG_PAYMENT'] = installments.groupby('SK_ID_PREV')['AMT_PAYMENT'].transform(lambda x: x.rolling(window=3, min_periods=1).mean())

# Cumulative Features
installments['TOTAL_PAID_SO_FAR'] = installments.groupby('SK_ID_PREV')['AMT_PAYMENT'].cumsum()

# Payment Regularity
installments['PAYMENT_REGULARITY'] = installments.groupby('SK_ID_PREV')['TIMING_DIFF'].transform('std')

# Delayed Payment Count
installments['DELAYED_PAYMENT_COUNT'] = (installments['TIMING_DIFF'] > 0).astype(int).groupby(installments['SK_ID_PREV']).cumsum()

# Interaction Features
installments['VERSION_PAYMENT_INTERACTION'] = installments['NUM_INSTALMENT_VERSION'] * installments['AMT_PAYMENT']

# Categorical Encoding of Version Changes
installments['VERSION_CHANGE_CAT'] = pd.Categorical(installments['VERSION_CHANGE'])


# Assuming installments is your DataFrame and DAYS_INSTALMENT is the relative day column
# Convert DAYS_INSTALMENT to a cumulative count
max_day = installments['DAYS_INSTALMENT'].abs().max()
installments['DAY_COUNT'] = max_day - installments['DAYS_INSTALMENT'].abs()

# Sort by this new count
installments = installments.sort_values(by=['SK_ID_PREV', 'DAY_COUNT'])

# Calculate the sum of payments for the last 180 days
# Here, 180 days is just an example, adjust as needed
installments['SUM_LAST_180_DAYS'] = installments.groupby('SK_ID_PREV')['AMT_PAYMENT'].rolling(window=180, min_periods=1).sum().reset_index(level=0, drop=True)

installments.head()

,SK_ID_PREV,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT,SK_ID_CURR,TARGET,VERSION_CHANGE,TIMING_DIFF,PAYMENT_RATIO,MOVING_AVG_PAYMENT,TOTAL_PAID_SO_FAR,PAYMENT_REGULARITY,DELAYED_PAYMENT_COUNT,VERSION_PAYMENT_INTERACTION,VERSION_CHANGE_CAT,DAY_COUNT,SUM_LAST_180_DAYS
0,1000001,1.0,1,-268.0,-294.0,6404.310,6404.310,117953.0,0.0,0.0,-26.0,1.0,6404.3100,6404.310,14.142136,0,6404.31,0.0,2654.0,6404.310
1,1000001,2.0,2,-238.0,-244.0,62039.115,62039.115,117953.0,0.0,1.0,-6.0,1.0,34221.7125,68443.425,14.142136,0,124078.23,1.0,2684.0,68443.425
3,1000003,1.0,1,-94.0,-108.0,4951.350,4951.350,6707.0,0.0,0.0,-14.0,1.0,4951.3500,9902.700,1.527525,0,4951.35,0.0,2828.0,4951.350
4,1000003,1.0,2,-64.0,-81.0,4951.350,4951.350,6707.0,0.0,0.0,-17.0,1.0,4951.3500,14854.050,1.527525,0,4951.35,0.0,2858.0,9902.700
2,1000003,1.0,3,-34.0,-49.0,4951.350,4951.350,6707.0,0.0,0.0,-15.0,1.0,4951.3500,4951.350,1.527525,0,4951.35,0.0,2888.0,14854.050


In [12]:
# fill na with mode
for col in installments.columns:
    installments[col].fillna(installments[col].mode()[0], inplace=True)

installments.isnull().sum()

SK_ID_PREV                     0
NUM_INSTALMENT_VERSION         0
NUM_INSTALMENT_NUMBER          0
DAYS_INSTALMENT                0
DAYS_ENTRY_PAYMENT             0
AMT_INSTALMENT                 0
AMT_PAYMENT                    0
SK_ID_CURR                     0
TARGET                         0
VERSION_CHANGE                 0
TIMING_DIFF                    0
PAYMENT_RATIO                  0
MOVING_AVG_PAYMENT             0
TOTAL_PAID_SO_FAR              0
PAYMENT_REGULARITY             0
DELAYED_PAYMENT_COUNT          0
VERSION_PAYMENT_INTERACTION    0
VERSION_CHANGE_CAT             0
DAY_COUNT                      0
SUM_LAST_180_DAYS              0
dtype: int64

aggregate

In [13]:
installments.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7744758 entries, 0 to 7744757
Data columns (total 20 columns):
 #   Column                       Dtype   
---  ------                       -----   
 0   SK_ID_PREV                   int64   
 1   NUM_INSTALMENT_VERSION       float64 
 2   NUM_INSTALMENT_NUMBER        int64   
 3   DAYS_INSTALMENT              float64 
 4   DAYS_ENTRY_PAYMENT           float64 
 5   AMT_INSTALMENT               float64 
 6   AMT_PAYMENT                  float64 
 7   SK_ID_CURR                   float64 
 8   TARGET                       float64 
 9   VERSION_CHANGE               float64 
 10  TIMING_DIFF                  float64 
 11  PAYMENT_RATIO                float64 
 12  MOVING_AVG_PAYMENT           float64 
 13  TOTAL_PAID_SO_FAR            float64 
 14  PAYMENT_REGULARITY           float64 
 15  DELAYED_PAYMENT_COUNT        int64   
 16  VERSION_PAYMENT_INTERACTION  float64 
 17  VERSION_CHANGE_CAT           category
 18  DAY_COUNT                  

In [14]:
aggregations_1 = {
    'SK_ID_CURR': ['first'],
    'NUM_INSTALMENT_VERSION': ['mean', 'sum', 'std'],
    'NUM_INSTALMENT_NUMBER': ['mean', 'sum', 'std'],
    'DAYS_INSTALMENT': ['mean', 'sum', 'std'],
    'DAYS_ENTRY_PAYMENT': ['mean', 'sum', 'std'],
    'AMT_INSTALMENT': ['mean', 'sum', 'std'],
    'AMT_PAYMENT': ['mean', 'sum', 'std'],
    'VERSION_CHANGE': ['mean', 'sum', 'std'],
    'TIMING_DIFF': ['mean', 'sum', 'std', 'max'],
    'PAYMENT_RATIO': ['mean', 'sum', 'std', 'min'],
    'MOVING_AVG_PAYMENT': ['mean', 'sum', 'std'],
    'TOTAL_PAID_SO_FAR': ['mean', 'sum', 'std'],
    'PAYMENT_REGULARITY': ['mean', 'sum', 'std'],
    'DELAYED_PAYMENT_COUNT': ['mean', 'sum', 'std'],
    'VERSION_PAYMENT_INTERACTION': ['mean', 'sum', 'std'],
    'VERSION_CHANGE_CAT': ['nunique'],
    'DAY_COUNT': ['mean', 'sum', 'std'],
    'SUM_LAST_180_DAYS': ['mean', 'sum', 'std']
}

installments_agg_1 = installments.groupby('SK_ID_PREV').agg(aggregations_1)
installments_agg_1.columns = pd.Index([e[0] + "_" + e[1].upper() for e in installments_agg_1.columns.tolist()])
installments_agg_1.head()

,SK_ID_CURR_FIRST,NUM_INSTALMENT_VERSION_MEAN,NUM_INSTALMENT_VERSION_SUM,NUM_INSTALMENT_VERSION_STD,NUM_INSTALMENT_NUMBER_MEAN,NUM_INSTALMENT_NUMBER_SUM,NUM_INSTALMENT_NUMBER_STD,DAYS_INSTALMENT_MEAN,DAYS_INSTALMENT_SUM,DAYS_INSTALMENT_STD,DAYS_ENTRY_PAYMENT_MEAN,DAYS_ENTRY_PAYMENT_SUM,DAYS_ENTRY_PAYMENT_STD,AMT_INSTALMENT_MEAN,AMT_INSTALMENT_SUM,AMT_INSTALMENT_STD,AMT_PAYMENT_MEAN,AMT_PAYMENT_SUM,AMT_PAYMENT_STD,VERSION_CHANGE_MEAN,VERSION_CHANGE_SUM,VERSION_CHANGE_STD,TIMING_DIFF_MEAN,TIMING_DIFF_SUM,TIMING_DIFF_STD,TIMING_DIFF_MAX,PAYMENT_RATIO_MEAN,PAYMENT_RATIO_SUM,PAYMENT_RATIO_STD,PAYMENT_RATIO_MIN,MOVING_AVG_PAYMENT_MEAN,MOVING_AVG_PAYMENT_SUM,MOVING_AVG_PAYMENT_STD,TOTAL_PAID_SO_FAR_MEAN,TOTAL_PAID_SO_FAR_SUM,TOTAL_PAID_SO_FAR_STD,PAYMENT_REGULARITY_MEAN,PAYMENT_REGULARITY_SUM,PAYMENT_REGULARITY_STD,DELAYED_PAYMENT_COUNT_MEAN,DELAYED_PAYMENT_COUNT_SUM,DELAYED_PAYMENT_COUNT_STD,VERSION_PAYMENT_INTERACTION_MEAN,VERSION_PAYMENT_INTERACTION_SUM,VERSION_PAYMENT_INTERACTION_STD,VERSION_CHANGE_CAT_NUNIQUE,DAY_COUNT_MEAN,DAY_COUNT_SUM,DAY_COUNT_STD,SUM_LAST_180_DAYS_MEAN,SUM_LAST_180_DAYS_SUM,SUM_LAST_180_DAYS_STD
SK_ID_PREV,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1000001,117953.0,1.500000,3.0,0.707107,1.500000,3,0.707107,-253.000000,-506.0,21.213203,-269.000000,-538.0,35.355339,34221.712500,68443.425,39339.747885,34221.712500,68443.425,39339.747885,0.500000,1.0,0.707107,-16.000000,-32.0,14.142136,-6.0,1.000000,2.0,0.000000,1.00000,20313.011250,40626.0225,19669.873943,37423.867500,74847.735,43868.278915,14.142136,28.284271,0.0,0.000000,0,0.0000,65241.270000,130482.540,83208.026801,2,2669.000000,5338.0,21.213203,37423.867500,74847.735,43868.278915
1000003,6707.0,1.000000,3.0,0.000000,2.000000,6,1.000000,-64.000000,-192.0,30.000000,-79.333333,-238.0,29.535290,4951.350000,14854.050,0.000000,4951.350000,14854.050,0.000000,0.000000,0.0,0.000000,-15.333333,-46.0,1.527525,-14.0,1.000000,3.0,0.000000,1.00000,4951.350000,14854.0500,0.000000,9902.700000,29708.100,4951.350000,1.527525,4.582576,0.0,0.000000,0,0.0000,4951.350000,14854.050,0.000000,1,2858.000000,8574.0,30.000000,9902.700000,29708.100,4951.350000
1000004,187620.0,1.142857,8.0,0.377964,4.000000,28,2.160247,-772.000000,-5404.0,64.807407,-798.714286,-5591.0,66.562538,4789.022143,33523.155,3698.527885,4789.022143,33523.155,3698.527885,0.142857,1.0,0.377964,-26.714286,-187.0,17.046156,-10.0,1.000000,7.0,0.000000,1.00000,3857.080714,26999.5650,1232.842628,14962.352143,104736.465,10026.332908,17.046156,119.323091,0.0,0.000000,0,0.0000,6671.378571,46699.650,8678.774874,2,2150.000000,15050.0,64.807407,14962.352143,104736.465,10026.332908
1000005,83759.0,1.000000,11.0,0.000000,5.818182,64,3.060006,-1543.454545,-16978.0,91.800178,-1551.909091,-17071.0,88.191218,14703.210000,161735.310,34.476315,13365.609545,147021.705,4432.077970,0.000000,0.0,0.000000,-8.454545,-93.0,10.289447,3.0,0.909091,10.0,0.301448,0.00019,13365.609545,147021.7050,2283.692611,84194.795455,926142.750,43642.847811,10.289447,113.183921,0.0,1.545455,17,0.8202,13365.609545,147021.705,4432.077970,1,1378.545455,15164.0,91.800178,85595.780455,941553.585,45007.812512
1000007,25033.0,1.000000,5.0,0.000000,3.000000,15,1.581139,-63.000000,-315.0,47.434165,-79.800000,-399.0,51.402335,11246.805000,56234.025,0.000000,11246.805000,56234.025,0.000000,0.000000,0.0,0.000000,-16.800000,-84.0,5.674504,-7.0,1.000000,5.0,0.000000,1.00000,11246.805000,56234.0250,0.000000,33740.415000,168702.075,17782.760100,5.674504,28.372522,0.0,0.000000,0,0.0000,11246.805000,56234.025,0.000000,1,2859.000000,14295.0,47.434165,33740.415000,168702.075,17782.760100


In [15]:
# rename Sk_ID_CURR_FIRST to SK_ID_CURR
installments_agg_1.rename(columns={'SK_ID_CURR_FIRST': 'SK_ID_CURR'}, inplace=True)
installments_agg_2 = installments_agg_1.groupby('SK_ID_CURR').agg(['mean', 'sum', 'std'])

# rename columns
installments_agg_2.columns = pd.Index([e[0] + "_" + e[1].upper() for e in installments_agg_2.columns.tolist()])

installments_agg_2.head()

,NUM_INSTALMENT_VERSION_MEAN_MEAN,NUM_INSTALMENT_VERSION_MEAN_SUM,NUM_INSTALMENT_VERSION_MEAN_STD,NUM_INSTALMENT_VERSION_SUM_MEAN,NUM_INSTALMENT_VERSION_SUM_SUM,NUM_INSTALMENT_VERSION_SUM_STD,NUM_INSTALMENT_VERSION_STD_MEAN,NUM_INSTALMENT_VERSION_STD_SUM,NUM_INSTALMENT_VERSION_STD_STD,NUM_INSTALMENT_NUMBER_MEAN_MEAN,NUM_INSTALMENT_NUMBER_MEAN_SUM,NUM_INSTALMENT_NUMBER_MEAN_STD,NUM_INSTALMENT_NUMBER_SUM_MEAN,NUM_INSTALMENT_NUMBER_SUM_SUM,NUM_INSTALMENT_NUMBER_SUM_STD,NUM_INSTALMENT_NUMBER_STD_MEAN,NUM_INSTALMENT_NUMBER_STD_SUM,NUM_INSTALMENT_NUMBER_STD_STD,DAYS_INSTALMENT_MEAN_MEAN,DAYS_INSTALMENT_MEAN_SUM,DAYS_INSTALMENT_MEAN_STD,DAYS_INSTALMENT_SUM_MEAN,DAYS_INSTALMENT_SUM_SUM,DAYS_INSTALMENT_SUM_STD,DAYS_INSTALMENT_STD_MEAN,DAYS_INSTALMENT_STD_SUM,DAYS_INSTALMENT_STD_STD,DAYS_ENTRY_PAYMENT_MEAN_MEAN,DAYS_ENTRY_PAYMENT_MEAN_SUM,DAYS_ENTRY_PAYMENT_MEAN_STD,DAYS_ENTRY_PAYMENT_SUM_MEAN,DAYS_ENTRY_PAYMENT_SUM_SUM,DAYS_ENTRY_PAYMENT_SUM_STD,DAYS_ENTRY_PAYMENT_STD_MEAN,DAYS_ENTRY_PAYMENT_STD_SUM,DAYS_ENTRY_PAYMENT_STD_STD,AMT_INSTALMENT_MEAN_MEAN,AMT_INSTALMENT_MEAN_SUM,AMT_INSTALMENT_MEAN_STD,AMT_INSTALMENT_SUM_MEAN,AMT_INSTALMENT_SUM_SUM,AMT_INSTALMENT_SUM_STD,AMT_INSTALMENT_STD_MEAN,AMT_INSTALMENT_STD_SUM,AMT_INSTALMENT_STD_STD,AMT_PAYMENT_MEAN_MEAN,AMT_PAYMENT_MEAN_SUM,AMT_PAYMENT_MEAN_STD,AMT_PAYMENT_SUM_MEAN,AMT_PAYMENT_SUM_SUM,...,TOTAL_PAID_SO_FAR_STD_SUM,TOTAL_PAID_SO_FAR_STD_STD,PAYMENT_REGULARITY_MEAN_MEAN,PAYMENT_REGULARITY_MEAN_SUM,PAYMENT_REGULARITY_MEAN_STD,PAYMENT_REGULARITY_SUM_MEAN,PAYMENT_REGULARITY_SUM_SUM,PAYMENT_REGULARITY_SUM_STD,PAYMENT_REGULARITY_STD_MEAN,PAYMENT_REGULARITY_STD_SUM,PAYMENT_REGULARITY_STD_STD,DELAYED_PAYMENT_COUNT_MEAN_MEAN,DELAYED_PAYMENT_COUNT_MEAN_SUM,DELAYED_PAYMENT_COUNT_MEAN_STD,DELAYED_PAYMENT_COUNT_SUM_MEAN,DELAYED_PAYMENT_COUNT_SUM_SUM,DELAYED_PAYMENT_COUNT_SUM_STD,DELAYED_PAYMENT_COUNT_STD_MEAN,DELAYED_PAYMENT_COUNT_STD_SUM,DELAYED_PAYMENT_COUNT_STD_STD,VERSION_PAYMENT_INTERACTION_MEAN_MEAN,VERSION_PAYMENT_INTERACTION_MEAN_SUM,VERSION_PAYMENT_INTERACTION_MEAN_STD,VERSION_PAYMENT_INTERACTION_SUM_MEAN,VERSION_PAYMENT_INTERACTION_SUM_SUM,VERSION_PAYMENT_INTERACTION_SUM_STD,VERSION_PAYMENT_INTERACTION_STD_MEAN,VERSION_PAYMENT_INTERACTION_STD_SUM,VERSION_PAYMENT_INTERACTION_STD_STD,VERSION_CHANGE_CAT_NUNIQUE_MEAN,VERSION_CHANGE_CAT_NUNIQUE_SUM,VERSION_CHANGE_CAT_NUNIQUE_STD,DAY_COUNT_MEAN_MEAN,DAY_COUNT_MEAN_SUM,DAY_COUNT_MEAN_STD,DAY_COUNT_SUM_MEAN,DAY_COUNT_SUM_SUM,DAY_COUNT_SUM_STD,DAY_COUNT_STD_MEAN,DAY_COUNT_STD_SUM,DAY_COUNT_STD_STD,SUM_LAST_180_DAYS_MEAN_MEAN,SUM_LAST_180_DAYS_MEAN_SUM,SUM_LAST_180_DAYS_MEAN_STD,SUM_LAST_180_DAYS_SUM_MEAN,SUM_LAST_180_DAYS_SUM_SUM,SUM_LAST_180_DAYS_SUM_STD,SUM_LAST_180_DAYS_STD_MEAN,SUM_LAST_180_DAYS_STD_SUM,SUM_LAST_180_DAYS_STD_STD
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0.0,1.008333,4.033333,0.683333,3.250000,13.0,2.500000,0.432918,1.731671,0.307497,4.275000,17.100000,4.262531,59.000000,236,102.127371,2.342536,9.370143,2.519231,-174.225000,-696.900000,133.984088,-1046.500000,-4186.0,892.563536,42.334654,169.338617,21.820181,-194.658333,-778.633333,147.253479,-1128.25,-4513.0,928.598361,46.285659,185.142637,23.073263,17514.131812,70056.527250,15919.817110,77735.59875,310942.395,44149.780173,21746.780365,86987.121460,19847.275901,17514.131812,70056.527250,15919.817110,77735.59875,310942.395,...,118840.928285,21502.535674,9.715769,38.863077,12.162081,56.001934,224.007736,59.820937,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0,0.00000,0.000000,0.000000,0.000000,28815.823125,115263.2925,31829.756403,91216.99125,364867.965,96917.351614,40910.913760,163643.655041,47457.120175,1.750000,7,0.500000,2747.775000,10991.100000,133.984088,20868.500000,83474.0,24011.566831,42.334654,169.338617,21.820181,35637.304125,142549.216500,19292.800471,3.353049e+05,1341219.645,4.668329e+05,34393.312986,137573.251946,21278.096586
1.0,0.686275,2.058824,0.595058,9.333333,28.0,9.018500,0.080845,0.242536

In [23]:
installments_agg_2.to_csv('processed-data/installments_agg_2.csv')

In [16]:
app_train = pd.read_csv('processed-data/app_train.csv')
app_train.set_index('SK_ID_CURR', inplace=True)
app_train.head()

,TARGET,CNT_CHILDREN,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,EXT_SOURCE_2,EXT_SOURCE_3,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_QRT,CODE_GENDER_F,CODE_GENDER_M,FLAG_OWN_CAR_N,FLAG_OWN_CAR_Y,NAME_INCOME_TYPE_Commercial associate,NAME_INCOME_TYPE_Other,NAME_INCOME_TYPE_Pensioner,NAME_INCOME_TYPE_Working,NAME_EDUCATION_TYPE_Higher education,NAME_EDUCATION_TYPE_Incomplete higher,NAME_EDUCATION_TYPE_Other,NAME_EDUCATION_TYPE_Secondary / secondary special,NAME_FAMILY_STATUS_Civil marriage,NAME_FAMILY_STATUS_Married,NAME_FAMILY_STATUS_Other,NAME_FAMILY_STATUS_Single / not married,NAME_HOUSING_TYPE_House / apartment,NAME_HOUSING_TYPE_Municipal apartment,NAME_HOUSING_TYPE_Other,NAME_HOUSING_TYPE_With parents,OCCUPATION_TYPE_Core staff,OCCUPATION_TYPE_Laborers,OCCUPATION_TYPE_Other,OCCUPATION_TYPE_Sales staff,ORGANIZATION_TYPE_Business Entity Type 3,ORGANIZATION_TYPE_Other,ORGANIZATION_TYPE_Self-employed
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,0,0,1197000.0,44487.0,1197000.0,0.026392,-11945,-376,-574.0,-580,1,0,0,0,2,2,0,0,0,0,0,0,0.126697,0.285180,0.0,0,1,0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
1,1,0,900000.0,26316.0,900000.0,0.003122,-19158,-9203,-12984.0,-2568,1,0,0,0,3,3,0,0,0,0,0,0,0.598301,0.746300,-142.0,0,1,0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
2,0,1,265851.0,11263.5,229500.0,0.031329,-14434,-3759,-4976.0,-3989,1,0,0,0,2,2,0,0,0,0,0,0,0.293988,0.415347,0.0,0,1,0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
3,0,2,545040.0,20547.0,450000.0,0.004849,-15957,-6018,-10110.0,-5219,1,0,1,0,2,2,0,0,0,0,0,0,0.070575,0.397946,-725.0,0,1,0,0.0,2.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
4,0,0,512064.0,25033.5,360000.0,0.018801,-17851,-495,-43.0,-181,1,0,0,0,2,2,0,0,0,0,1,1,0.501790,0.529890,0.0,0,1,0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0


In [18]:
# merge train data with installments
app_train = app_train.merge(installments_agg_2, left_index=True, right_index=True, how='left')
app_train.head()


,TARGET,CNT_CHILDREN,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,EXT_SOURCE_2,EXT_SOURCE_3,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_QRT,CODE_GENDER_F,CODE_GENDER_M,FLAG_OWN_CAR_N,FLAG_OWN_CAR_Y,NAME_INCOME_TYPE_Commercial associate,NAME_INCOME_TYPE_Other,NAME_INCOME_TYPE_Pensioner,NAME_INCOME_TYPE_Working,NAME_EDUCATION_TYPE_Higher education,NAME_EDUCATION_TYPE_Incomplete higher,NAME_EDUCATION_TYPE_Other,NAME_EDUCATION_TYPE_Secondary / secondary special,NAME_FAMILY_STATUS_Civil marriage,NAME_FAMILY_STATUS_Married,NAME_FAMILY_STATUS_Other,NAME_FAMILY_STATUS_Single / not married,NAME_HOUSING_TYPE_House / apartment,NAME_HOUSING_TYPE_Municipal apartment,NAME_HOUSING_TYPE_Other,NAME_HOUSING_TYPE_With parents,...,TOTAL_PAID_SO_FAR_STD_SUM,TOTAL_PAID_SO_FAR_STD_STD,PAYMENT_REGULARITY_MEAN_MEAN,PAYMENT_REGULARITY_MEAN_SUM,PAYMENT_REGULARITY_MEAN_STD,PAYMENT_REGULARITY_SUM_MEAN,PAYMENT_REGULARITY_SUM_SUM,PAYMENT_REGULARITY_SUM_STD,PAYMENT_REGULARITY_STD_MEAN,PAYMENT_REGULARITY_STD_SUM,PAYMENT_REGULARITY_STD_STD,DELAYED_PAYMENT_COUNT_MEAN_MEAN,DELAYED_PAYMENT_COUNT_MEAN_SUM,DELAYED_PAYMENT_COUNT_MEAN_STD,DELAYED_PAYMENT_COUNT_SUM_MEAN,DELAYED_PAYMENT_COUNT_SUM_SUM,DELAYED_PAYMENT_COUNT_SUM_STD,DELAYED_PAYMENT_COUNT_STD_MEAN,DELAYED_PAYMENT_COUNT_STD_SUM,DELAYED_PAYMENT_COUNT_STD_STD,VERSION_PAYMENT_INTERACTION_MEAN_MEAN,VERSION_PAYMENT_INTERACTION_MEAN_SUM,VERSION_PAYMENT_INTERACTION_MEAN_STD,VERSION_PAYMENT_INTERACTION_SUM_MEAN,VERSION_PAYMENT_INTERACTION_SUM_SUM,VERSION_PAYMENT_INTERACTION_SUM_STD,VERSION_PAYMENT_INTERACTION_STD_MEAN,VERSION_PAYMENT_INTERACTION_STD_SUM,VERSION_PAYMENT_INTERACTION_STD_STD,VERSION_CHANGE_CAT_NUNIQUE_MEAN,VERSION_CHANGE_CAT_NUNIQUE_SUM,VERSION_CHANGE_CAT_NUNIQUE_STD,DAY_COUNT_MEAN_MEAN,DAY_COUNT_MEAN_SUM,DAY_COUNT_MEAN_STD,DAY_COUNT_SUM_MEAN,DAY_COUNT_SUM_SUM,DAY_COUNT_SUM_STD,DAY_COUNT_STD_MEAN,DAY_COUNT_STD_SUM,DAY_COUNT_STD_STD,SUM_LAST_180_DAYS_MEAN_MEAN,SUM_LAST_180_DAYS_MEAN_SUM,SUM_LAST_180_DAYS_MEAN_STD,SUM_LAST_180_DAYS_SUM_MEAN,SUM_LAST_180_DAYS_SUM_SUM,SUM_LAST_180_DAYS_SUM_STD,SUM_LAST_180_DAYS_STD_MEAN,SUM_LAST_180_DAYS_STD_SUM,SUM_LAST_180_DAYS_STD_STD
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0.0,0,0,1197000.0,44487.0,1197000.0,0.026392,-11945,-376,-574.0,-580,1,0,0,0,2,2,0,0,0,0,0,0,0.126697,0.285180,0.0,0,1,0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,118840.928285,21502.535674,9.715769,38.863077,12.162081,56.001934,224.007736,59.820937,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.00000,0.000000,0.000000,0.000000,28815.823125,115263.2925,31829.756403,91216.99125,364867.965,96917.351614,40910.913760,163643.655041,47457.120175,1.750000,7.0,0.500000,2747.775000,10991.100000,133.984088,20868.500000,83474.0,24011.566831,42.334654,169.338617,21.820181,35637.304125,142549.216500,19292.800471,3.353049e+05,1341219.645,4.668329e+05,34393.312986,137573.251946,21278.096586
1.0,1,0,900000.0,26316.0,900000.0,0.003122,-19158,-9203,-12984.0,-2568,1,0,0,0,3,3,0,0,0,0,0,0,0.598301,0.746300,-142.0,0,1,0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,190767.494424,66717.731820,7.685104,23.055312,8.391806,119.049743,357.149228,151.788968,0.0,0.0,0.0,2.083107,6.249321,1.463244,31.0,93.0,28.79236,1.248148,3.744445,0.772467,22754.550000,68263.6500,30785.973981,362376.00000,1087128.000,539452.111673,57229.671951,171689.015853,99119.770057,1.333333,4.0,0.577350,2227.749623,6683.248869,688.985228,30691.666667,92075.0,14017.037645,85.006509,255

In [19]:
app_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 246009 entries, 0.0 to 307510.0
Columns: 210 entries, TARGET to SUM_LAST_180_DAYS_STD_STD
dtypes: float64(190), int64(20)
memory usage: 404.1 MB


In [20]:
# check inf values
app_train.replace([np.inf, -np.inf], np.nan, inplace=True)
app_train.isnull().sum().sum()

# fill na with 0
for col in app_train.columns:
    app_train[col].fillna(0, inplace=True)
app_train.isnull().sum().sum()

0

In [21]:
X, y = app_train.drop('TARGET', axis=1), app_train['TARGET']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# scale data
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [24]:
logreg = LogisticRegression(class_weight='balanced', solver='newton-cholesky')
logreg.fit(X_train, y_train)

y_pred = logreg.predict(X_test)
y_pred_proba = logreg.predict_proba(X_test)[:, 1]

gini(y_test, y_pred_proba)

0.4930793673970708